In [1]:
import os

import mlflow

/Users/nikolaistepanov/.pyenv/versions/3.11.3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/nikolaistepanov/.pyenv/versions/3.11.3/lib/python3.11/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


##### 1. Поднимаем MLFlow tracking server локально c Postgres & S3 Object Storage

**Важно** - `credentials` тут мы передаем для нашего **tracking service**. То есть чтобы у него был доступ к базе данных и бакету

###### Запускаем команду при помощи: `sh utils/1_run_mlflow_server.sh`


###### Сама команда выглядит следующим образом: 
```shell
export MLFLOW_S3_ENDPOINT_URL=...
export AWS_ACCESS_KEY_ID=...
export AWS_SECRET_ACCESS_KEY=...

mlflow server \
    --backend-store-uri postgresql://{user}:{password}@{host}:{port}/{database} \
	--registry-store-uri postgresql://{user}:{password}@{host}:{port}/{database} \
	--default-artifact-root s3://{bucket} \
	--no-serve-artifacts
```

##### 1. Определяем основные credentials, которые нам нужны для подключения к MLFlow

**Важно** - `credentials` тут мы передаем для нас, как пользователя tracking service. То есть чтобы у **нас** был доступ к бакету, в который мы будем складывать артефакты

In [5]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "..." # endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = "..." # внесите id ключа бакета, к которому подключен MLFlow
os.environ["AWS_SECRET_ACCESS_KEY"] = "..."  # внесите ключ бакета, к которому подключен MLFlow

##### 2. Определяем глобальные переменные

In [11]:
# поднимаем MLFlow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# название тестового experiment и run внутри него
EXPERIMENT_NAME = "test_connection_experiment_nikolaistepanov"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

##### 2. Устанавливаем host, который будет трекать наши эксперименты

In [12]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

##### 3. Создаем тестовый эксперимент и записываем в него тестовую информацию

In [13]:
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

In [14]:
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE)

##### 4. Проверяем себя, что в MLFlow:
- создался `experiment` с нашим именем
- внутри эксперимента появился `run`
- внутри `run` записалась наша тестовая `metric`

In [15]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
run = mlflow.get_run(run_id)

In [17]:
assert "active" == experiment.lifecycle_stage
assert mlflow.get_run(run_id)
assert METRIC_VALUE == run.data.metrics[METRIC_NAME]